## Load in the bridge dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bridges_full = pd.read_excel('../data/raw/BMMS_overview.xlsx')
bridges_full

road      km                   type  LRPName               name  \
0         N1   1.800            Box Culvert  LRP001a                  .   
1         N1   4.925            Box Culvert  LRP004b                  .   
2         N1   8.976       PC Girder Bridge  LRP008b  Kanch pur Bridge.   
3         N1  10.880            Box Culvert  LRP010b   NOYAPARA CULVERT   
4         N1  10.897            Box Culvert  LRP010c     ADUPUR CULVERT   
...      ...     ...                    ...      ...                ...   
20410  Z8806  83.728            Box Culvert  LRP083a                  .   
20411  Z8810   3.981  Steel Beam & RCC Slab  LRP003b                  .   
20412  Z8814  20.730  Baily with Steel Deck  LRP020b     Kalbari Bridge   
20413  Z8910  28.880      RCC Girder Bridge  LRP028a   Narainpur Bridge   
20414  Z8910  33.530      RCC Girder Bridge  LRP033c       Jor Bridge 2   

       length condition  structureNr  \
0       11.30         A       117861   
1        6.60         A       117862   
2      394.23         A       119889   
3        6.30         A       112531   
4        6.30         A       112532   
...       ...       ...          ...   
20410    1.78         D       110470   
20411   27.20         D       113458   
20412   30.50         D       121147   
20413    9.50         D       119249   
20414    6.00         D       119254   

                                                roadName  chainage  width  \
0      Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     1.800   19.5   
1      Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     4.925   35.4   
2      Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     8.976    NaN   
3      Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    10.880   12.2   
4      Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    10.897   12.2   
...                                                  ...       ...    ...   
20410  Lebukhali-Dumki-Boga-Bauphal-Kalaiya-Dashmina-...    83.728    7.6   
20411  Bakerganj-Padrishibpur-Kathaltali-Subidkhali-B...     3.981    3.2   
20412  Barguna (Dakhin Ramna Kheya Ghat)-Dewatala-Mat...    20.730    NaN   
20413       Barisal (Bairagirpool)-Tumchar-Bauphaul Road    28.880    NaN   
20414       Barisal (Bairagirpool)-Tumchar-Bauphaul Road    33.530    NaN   

       constructionYear  spans     zone      circle     division  \
0                2005.0    2.0    Dhaka       Dhaka  Narayanganj   
1                2006.0    1.0    Dhaka       Dhaka  Narayanganj   
2                   NaN    NaN    Dhaka       Dhaka  Narayanganj   
3                1992.0    2.0    Dhaka       Dhaka  Narayanganj   
4                1984.0    2.0    Dhaka       Dhaka  Narayanganj   
...                 ...    ...      ...         ...          ...   
20410            1980.0    1.0  Barisal  Patuakhali      Barguna   
20411            1979.0    5.0  Barisal     Barisal      Barisal   
20412               NaN    NaN  Barisal     Barisal     Pirojpur   
20413               NaN    NaN  Barisal  Patuakhali   Patuakhali   
20414               NaN    NaN  Barisal  Patuakhali   Patuakhali   

        sub-division        lat        lon EstimatedLoc  
0      Narayanganj-1  23.698739  90.458861  interpolate  
1      Narayanganj-1  23.694664  90.487775  interpolate  
2      Narayanganj-1  23.705060  90.523214  interpolate  
3          Vitikandi  23.694391  90.537574  interpolate  
4          Vitikandi  23.694302  90.537707  interpolate  
...              ...        ...        ...          ...  
20410        Barguna  22.183448  90.299962  interpolate  
20411      Barisal-1  22.519353  90.310063  interpolate  
20412       Kawkhali  22.282704  89.968512  interpolate  
20413     Patuakhali  22.510650  90.535314  interpolate  
20414     Patuakhali  22.474325  90.557335  interpolate  

[20415 rows x 20 columns]

In [3]:
# Verify that there are no duplicate structure numbers 
bridges_full[bridges_full.duplicated('structureNr')]

Empty DataFrame
Columns: [road, km, type, LRPName, name, length, condition, structureNr, roadName, chainage, width, constructionYear, spans, zone, circle, division, sub-division, lat, lon, EstimatedLoc]
Index: []

In [4]:
# From the previous assignment we know that bridges that are duplicated on either LRPName or coordinates, are mostly the same on other values as well. Some have minimal differences on bridge length or have missing values. Because of this, this function identifies duplicates based on only LRPName and chainage, for latter of which has been added as addittional check. 
def find_dupes_LRP(dataframe):
    df_copy = dataframe.copy()
    df_copy.sort_values(by=['LRPName', 'width'], inplace=True) #sort in such a way that the NaN values will be the bottom row
    df_copy.reset_index(inplace=True)
    # Check how many times each of the LRPs occur in the dataframe
    LRP_values = df_copy['LRPName'].value_counts()
    LRP_list = list(set((df_copy['LRPName'].to_list())))
    to_be_removed = []
    
    # For each LRP in the list, check if it has a duplicate, if so delete the duplicates while making sure that the worst bridge condition is kept
    for LRP in LRP_list:
        if LRP_values[LRP] == 2:
            bridge1_index = df_copy.index.get_loc(df_copy[df_copy['LRPName'] == LRP].index[0])
            if df_copy.loc[bridge1_index, 'chainage'] == df_copy.loc[bridge1_index+1, 'chainage']: 
                con_bridge1 = df_copy.loc[bridge1_index, 'condition']
                con_bridge2 = df_copy.loc[bridge1_index+1, 'condition']
                if con_bridge1 < con_bridge2:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge2
                to_be_removed.append(df_copy.loc[bridge1_index+1, 'structureNr'])
            
        if LRP_values[LRP] == 3:
            bridge1_index = df_copy.index.get_loc(df_copy[df_copy['LRPName'] == LRP].index[0])
            if df_copy.loc[bridge1_index, 'chainage'] == df_copy.loc[bridge1_index+1, 'chainage'] == df_copy.loc[bridge1_index+2, 'chainage']:
                con_bridge1 = df_copy.loc[bridge1_index, 'condition']
                con_bridge2 = df_copy.loc[bridge1_index+1, 'condition']
                con_bridge3 = df_copy.loc[bridge1_index+2, 'condition']
                if con_bridge1 < con_bridge2:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge2
                if con_bridge1 < con_bridge3:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge3
                to_be_removed.append(df_copy.loc[bridge1_index+1, 'structureNr'])
                to_be_removed.append(df_copy.loc[bridge1_index+2, 'structureNr'])
            
        if LRP_values[LRP] == 4:
            bridge1_index = df_copy.index.get_loc(df_copy[df_copy['LRPName'] == LRP].index[0])
            if df_copy.loc[bridge1_index, 'chainage'] == df_copy.loc[bridge1_index+1, 'chainage'] == df_copy.loc[bridge1_index+2, 'chainage'] == df_copy.loc[bridge1_index+3, 'chainage']:
                con_bridge1 = df_copy.loc[bridge1_index, 'condition']
                con_bridge2 = df_copy.loc[bridge1_index+1, 'condition']
                con_bridge3 = df_copy.loc[bridge1_index+2, 'condition']
                con_bridge4 = df_copy.loc[bridge1_index+3, 'condition']
                if con_bridge1 < con_bridge2:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge2
                if con_bridge1 < con_bridge3:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge3
                if con_bridge1 < con_bridge4:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge4
                to_be_removed.append(df_copy.loc[bridge1_index+1, 'structureNr'])
                to_be_removed.append(df_copy.loc[bridge1_index+2, 'structureNr'])
                to_be_removed.append(df_copy.loc[bridge1_index+3, 'structureNr'])
    
    # Make a new dataframe containing only the cleaned bridges, and the ones that do not have a dupe
    df_cleaned = df_copy[~df_copy['structureNr'].isin(to_be_removed)]
    return df_cleaned

In [5]:
# Make a smaller dataframe to test if the code works the way it is supposed to
bridges_N1 = bridges_full[bridges_full['road']=='N1']
bridges_N1sort = bridges_N1.sort_values(by='LRPName')
bridges_N1sort.reset_index(inplace=True)
LRP_valuesN1 = bridges_N1sort['LRPName'].value_counts()
bridge_LRPN1 = bridges_N1sort[bridges_N1sort['LRPName'].isin(LRP_valuesN1[LRP_valuesN1>1].index)]
bridge_slice = bridge_LRPN1.head(8)
bridge_slice

index road      km               type  LRPName  \
2       2   N1   8.976   PC Girder Bridge  LRP008b   
3   12706   N1   8.976   PC Girder Bridge  LRP008b   
11      9   N1  12.660   PC Girder Bridge  LRP013a   
12     10   N1  12.660   PC Girder Bridge  LRP013a   
13      8   N1  12.660   PC Girder Bridge  LRP013a   
17  14588   N1  17.134   PC Girder Bridge  LRP017b   
18  12707   N1  17.134  RCC Girder Bridge  LRP017b   
19  14590   N1  17.722   PC Girder Bridge  LRP018a   

                           name  length condition  structureNr  \
2             Kanch pur Bridge.  394.23         A       119889   
3     KANCHPUR PC GIRDER BRIDGE  397.00         C       101102   
11                MADAN PUR (R)   26.30         A       109841   
12           MADANPUR BRIDGE(L)   26.30         A       109838   
13          Madanpur Bridge.(L)   27.50         A       119897   
17  LANGOLBANDO PC GIDER BRIDGE  159.50         B       109808   
18          Langalbandh Bridge.  159.52         C       119909   
19         DARIKANDI BRIDGE (L)   20.00         B       109853   

                                             roadName  ...  width  \
2   Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...    NaN   
3   Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...  14.65   
11  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...   9.20   
12  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...   9.20   
13  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...    NaN   
17  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...  17.80   
18  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...    NaN   
19  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...   9.20   

    constructionYear  spans   zone circle     division   sub-division  \
2                NaN    NaN  Dhaka  Dhaka  Narayanganj  Narayanganj-1   
3             1986.0    8.0  Dhaka  Dhaka  Narayanganj  Narayanganj-1   
11            2003.0    1.0  Dhaka  Dhaka  Narayanganj      Vitikandi   
12            2003.0    1.0  Dhaka  Dhaka  Narayanganj      Vitikandi   
13               NaN    NaN  Dhaka  Dhaka  Narayanganj      Vitikandi   
17            1998.0    5.0  Dhaka  Dhaka  Narayanganj      Vitikandi   
18               NaN    NaN  Dhaka  Dhaka  Narayanganj      Vitikandi   
19            2002.0    1.0  Dhaka  Dhaka  Narayanganj      Vitikandi   

          lat        lon  EstimatedLoc  
2   23.705060  90.523214   interpolate  
3   23.705060  90.523214   interpolate  
11  23.685583  90.551208   interpolate  
12  23.685583  90.551208   interpolate  
13  23.685583  90.551208   interpolate  
17  23.654426  90.576730   interpolate  
18  23.654426  90.576730   interpolate  
19  23.651571  90.581494   interpolate  

[8 rows x 21 columns]

In [6]:
df_slice_clean = find_dupes_LRP(bridge_slice)
df_slice_clean

level_0  index road      km              type  LRPName  \
0        3  12706   N1   8.976  PC Girder Bridge  LRP008b   
2       11      9   N1  12.660  PC Girder Bridge  LRP013a   
5       17  14588   N1  17.134  PC Girder Bridge  LRP017b   
7       19  14590   N1  17.722  PC Girder Bridge  LRP018a   

                          name  length condition  structureNr  ...  width  \
0    KANCHPUR PC GIRDER BRIDGE   397.0         C       101102  ...  14.65   
2                MADAN PUR (R)    26.3         A       109841  ...   9.20   
5  LANGOLBANDO PC GIDER BRIDGE   159.5         C       109808  ...  17.80   
7         DARIKANDI BRIDGE (L)    20.0         B       109853  ...   9.20   

   constructionYear  spans   zone  circle     division   sub-division  \
0            1986.0    8.0  Dhaka   Dhaka  Narayanganj  Narayanganj-1   
2            2003.0    1.0  Dhaka   Dhaka  Narayanganj      Vitikandi   
5            1998.0    5.0  Dhaka   Dhaka  Narayanganj      Vitikandi   
7            2002.0    1.0  Dhaka   Dhaka  Narayanganj      Vitikandi   

         lat        lon  EstimatedLoc  
0  23.705060  90.523214   interpolate  
2  23.685583  90.551208   interpolate  
5  23.654426  90.576730   interpolate  
7  23.651571  90.581494   interpolate  

[4 rows x 22 columns]

The code is correct: only one of the duplicated bridges is kept, with the worst condition of the lot. In addition, the dataframe is sorted in such a way, that the kept bridge has a minimal amount of NaN-values. Lastly, LRPs that do not have a dupe, are not touched and therefor remain in the dataframe as such

## Start cleaning the bridges on the identified roads

In [7]:
roads_df = pd.read_csv('../data/semi-processed/identified_roads_final.csv')
roads_list = roads_df['road'].to_list()
roads = set(roads_list)
len(roads)

26

In [8]:
# To get an insight into how many bridges have been removed, check the original amount of bridges
bridges_df_list = []
for road in roads:
    dataframe = bridges_full[bridges_full['road'] == road]
    bridges_df_list.append(dataframe)
    df_bridges_full = pd.concat(bridges_df_list)

df_bridges_full

road      km                   type  LRPName  \
5626   Z1044   1.149            Box Culvert  LRP001a   
5627   Z1044   1.747            Box Culvert  LRP001b   
5628   Z1044   4.101            Box Culvert  LRP004b   
5629   Z1044   4.300            Box Culvert  LRP004c   
5630   Z1044   4.823      RCC Girder Bridge  LRP004d   
...      ...     ...                    ...      ...   
16418   R240  42.428            Box Culvert  LRP042c   
16419   R240  43.620            Box Culvert  LRP043b   
16420   R240  46.849            Box Culvert  LRP047a   
16421   R240  48.270  Baily with Steel Deck  LRP048a   
19508   R240  45.180            Box Culvert  LRP045a   

                                name  length condition  structureNr  \
5626                amirabad box cul     4.5         A       102563   
5627         ETACOLA XOX CUL CULVERT     4.6         A       102565   
5628               MOLOBAZAR CULVERT     6.0         A       102573   
5629                       MOLOBAZAR     0.9         A       109303   
5630          MOHAMMADPUR BRIDGE (2)     7.5         A       102577   
...                              ...     ...       ...          ...   
16418                    SALAMAT PUR     3.0         C       102267   
16419           BADSHARA BOX CULVERT     3.0         C       102252   
16420            KHURSIB BOX CULVERT     3.1         C       102239   
16421  PAIPUR BAILEY WITH STEEL DECK    24.3         C       102247   
19508        BANGLABAZAR BOX CULVERT    18.9         D       102230   

                                                roadName  chainage  width  \
5626                       Gouripur-Kachua-Hajiganj Road     1.149   7.20   
5627                       Gouripur-Kachua-Hajiganj Road     1.747   9.75   
5628                       Gouripur-Kachua-Hajiganj Road     4.101   9.70   
5629                       Gouripur-Kachua-Hajiganj Road     4.300   5.66   
5630                       Gouripur-Kachua-Hajiganj Road     4.823   8.20   
...                                                  ...       ...    ...   
16418  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    42.428  11.85   
16419  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    43.620   9.80   
16420  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    46.849   9.70   
16421  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    48.270   3.80   
19508  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    45.180   9.80   

       constructionYear  spans     zone         circle  division sub-division  \
5626             1989.0    1.0  Comilla        Comilla   Comilla     Gouripur   
5627             1989.0    1.0  Comilla        Comilla   Comilla     Gouripur   
5628             1992.0    1.0  Comilla        Comilla   Comilla     Gouripur   
5629             1992.0    1.0  Comilla        Comilla   Comilla     Gouripur   
5630             1995.0    1.0  Comilla        Comilla   Comilla     Gouripur   
...                 ...    ...      ...            ...       ...          ...   
16418            1998.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
16419            2004.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
16420            1998.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
16421            2002.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
19508            1980.0    3.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   

             lat        lon EstimatedLoc  
5626   23.520553  90.794156  interpolate  
5627   23.515276  90.794962  interpolate  
5628   23.495697  90.797890  interpolate  
5629   23.495603  90.799416  interpolate  
5630   23.496520  90.804318  interpolate  
...          ...        ...          ...  
16418  24.572295  91.536863  interpolate  
16419  24.576429  91.547867  interpolate  
16420  24.587043  91.577156  interpolate  
16421  24.590992  91.590435  interpolate  
19508  24.579136  91.563132  interpolate  

[2862 rows x 20 columns]

In [9]:
df_bridges_full.isna().sum()

road                  0
km                    0
type                  0
LRPName               0
name                  0
length                1
condition             0
structureNr           0
roadName              0
chainage              0
width               463
constructionYear    463
spans               463
zone                  0
circle                0
division              0
sub-division          0
lat                   0
lon                   0
EstimatedLoc          0
dtype: int64

In [10]:
def correct_dupes_LRP(df, road_list):
    # Empty list to save in each cleaned dataframe per road
    full_cleaned_df_list = []
    
    # For each road, go trough the find_dupes_LRP function
    for road in road_list:
        dataframe = df[df['road'] == road]
        dataframe_cleaned = find_dupes_LRP(dataframe)
        full_cleaned_df_list.append(dataframe_cleaned)
    
    # Once the bridges on all the roads have been cleaned, concat the seperate road dataframes into one big dataframe again    
    df_cleaned = pd.concat(full_cleaned_df_list)
    return df_cleaned

In [11]:
cleaned_bridges_LRP = correct_dupes_LRP(bridges_full, roads)
cleaned_bridges_LRP

index   road      km                   type  LRPName  \
0    5626  Z1044   1.149            Box Culvert  LRP001a   
1    5627  Z1044   1.747            Box Culvert  LRP001b   
2   11349  Z1044   2.358      RCC Girder Bridge  LRP002b   
4   19772  Z1044   3.112      RCC Girder Bridge  LRP003a   
5    5628  Z1044   4.101            Box Culvert  LRP004b   
..    ...    ...     ...                    ...      ...   
68  15365   R240  47.812            Box Culvert  LRP047e   
69  16421   R240  48.270  Baily with Steel Deck  LRP048a   
71   2844   R240  48.610            Box Culvert  LRP048c   
72  15366   R240  49.160            Box Culvert  LRP049a   
73   2845   R240  49.333            Box Culvert  LRP049c   

                             name  length condition  structureNr  \
0                amirabad box cul    4.50         A       102563   
1         ETACOLA XOX CUL CULVERT    4.60         A       102565   
2                MAHANANDA BRIDGE   17.80         C       102567   
4             SAISTANAGATR BRIDGE    3.60         D       102570   
5               MOLOBAZAR CULVERT    6.00         A       102573   
..                            ...     ...       ...          ...   
68             KHURSI BOX CULVERT   12.23         B       102244   
69  PAIPUR BAILEY WITH STEEL DECK   24.30         C       102247   
71             RAIPUR BOX CULVERT    1.50         A       102249   
72           AUSKANDI BOX CULVERT   12.30         B       102250   
73           AUSKAMDI BOX CULVERT    1.50         A       102253   

                                             roadName  ...  width  \
0                       Gouripur-Kachua-Hajiganj Road  ...   7.20   
1                       Gouripur-Kachua-Hajiganj Road  ...   9.75   
2                       Gouripur-Kachua-Hajiganj Road  ...   9.92   
4                       Gouripur-Kachua-Hajiganj Road  ...   4.85   
5                       Gouripur-Kachua-Hajiganj Road  ...   9.70   
..                                                ...  ...    ...   
68  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   9.75   
69  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   3.80   
71  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   8.50   
72  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   9.75   
73  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   8.50   

    constructionYear  spans     zone         circle  division sub-division  \
0             1989.0    1.0  Comilla        Comilla   Comilla     Gouripur   
1             1989.0    1.0  Comilla        Comilla   Comilla     Gouripur   
2             1992.0    1.0  Comilla        Comilla   Comilla     Gouripur   
4             1967.0    1.0  Comilla        Comilla   Comilla     Gouripur   
5             1992.0    1.0  Comilla        Comilla   Comilla     Gouripur   
..               ...    ...      ...            ...       ...          ...   
68            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
69            2002.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
71            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
72            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
73            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   

          lat        lon  EstimatedLoc  
0   23.520553  90.794156   interpolate  
1   23.515276  90.794962   interpolate  
2   23.510033  90.796214   interpolate  
4   23.503492  90.796684   interpolate  
5   23.495697  90.797890   interpolate  
..        ...        ...           ...  
68  24.590565  91.585901   interpolate  
69  24.590992  91.590435   interpolate  
71  24.590976  91.592102   interpolate  
72  24.591276  91.599333   interpolate  
73  24.591425  91.601015   interpolate  

[2399 rows x 21 columns]

In [12]:
cleaned_bridges_LRP.isna().sum()

index                0
road                 0
km                   0
type                 0
LRPName              0
name                 0
length               0
condition            0
structureNr          0
roadName             0
chainage             0
width               30
constructionYear    30
spans               30
zone                 0
circle               0
division             0
sub-division         0
lat                  0
lon                  0
EstimatedLoc         0
dtype: int64

In this first iteration of cleaning, 427 duplicated bridges got removed. With this, the amount of missing values also greatly decreased, from 426 entries with missing values to only 21. Duplicates can also occur if bridges are put on exactly the same place, but do not have the same LRP. These duplicates will also be handeled 

## Check for dupes based on coordinates

In [13]:
lat_values = cleaned_bridges_LRP['lat'].value_counts()
lat_values.value_counts()

count
1    2371
2      14
Name: count, dtype: int64

In [14]:
lon_values = cleaned_bridges_LRP['lon'].value_counts()
lon_values.value_counts()

count
1    2368
2      14
3       1
Name: count, dtype: int64

As there are duplicates in the coordinates, repeat the same steps as before. 

In [15]:
def find_dupes_coord(dataframe):
    df_copy = dataframe.copy()
    df_copy.sort_values(by=['lat', 'width'], inplace=True) #sort that the matching coordinates will be paired
    df_copy.reset_index(inplace=True)
    to_be_removed = []
    lat_values = df_copy['lat'].value_counts()
    lat_list = list(set((df_copy['lat'].to_list())))
    
    for lat in lat_list:
        if lat_values[lat] == 2:
            # For each duped latitude, check if the londitude also matches, before continuing
            bridge1_index = df_copy.index.get_loc(df_copy[df_copy['lat'] == lat].index[0])
            if df_copy.loc[bridge1_index, 'lon'] == df_copy.loc[bridge1_index+1, 'lon']: 
                con_bridge1 = df_copy.loc[bridge1_index, 'condition']
                con_bridge2 = df_copy.loc[bridge1_index+1, 'condition']
                if con_bridge1 < con_bridge2:
                    df_copy.at[bridge1_index, 'condition'] = con_bridge2
                to_be_removed.append(df_copy.loc[bridge1_index+1, 'structureNr'])
            
    df_cleaned = df_copy[~df_copy['structureNr'].isin(to_be_removed)]
    return df_cleaned

In [16]:
def correct_dupes_coord(df, road_list):
    full_cleaned_df_list = []
    
    for road in road_list:
        dataframe = df[df['road'] == road]
        dataframe_cleaned = find_dupes_coord(dataframe)
        full_cleaned_df_list.append(dataframe_cleaned)
        
    df_cleaned = pd.concat(full_cleaned_df_list)
    return df_cleaned

In [17]:
df_cleaned_coord = correct_dupes_coord(cleaned_bridges_LRP, roads)
df_cleaned_coord

level_0  index   road      km                   type  LRPName  \
0        70   5675  Z1044  39.882            Box Culvert  LRP040b   
1        69   5674  Z1044  39.444            Box Culvert  LRP040a   
2        68   5673  Z1044  38.698       PC Girder Bridge  LRP039c   
3        67   5672  Z1044  38.530            Box Culvert  LRP039b   
4        66   5671  Z1044  38.271            Box Culvert  LRP039a   
..      ...    ...    ...     ...                    ...      ...   
60       68  15365   R240  47.812            Box Culvert  LRP047e   
61       71   2844   R240  48.610            Box Culvert  LRP048c   
62       69  16421   R240  48.270  Baily with Steel Deck  LRP048a   
63       72  15366   R240  49.160            Box Culvert  LRP049a   
64       73   2845   R240  49.333            Box Culvert  LRP049c   

                             name  length condition  structureNr  ... width  \
0            HOZIGANG BOX CULVERT   12.30         A       102609  ...  7.20   
1               BODORPUR SUBMARGE    1.50         A       102607  ...  5.66   
2                 BODORPUR BRIDGE   64.60         A       102608  ...  9.40   
3        kathalia sub box culvert    1.50         A       102604  ...  5.66   
4                KATHALIA CULVERT    3.00         A       102602  ...  7.30   
..                            ...     ...       ...          ...  ...   ...   
60             KHURSI BOX CULVERT   12.23         B       102244  ...  9.75   
61             RAIPUR BOX CULVERT    1.50         A       102249  ...  8.50   
62  PAIPUR BAILEY WITH STEEL DECK   24.30         C       102247  ...  3.80   
63           AUSKANDI BOX CULVERT   12.30         B       102250  ...  9.75   
64           AUSKAMDI BOX CULVERT    1.50         A       102253  ...  8.50   

    constructionYear  spans     zone         circle  division sub-division  \
0             2001.0    2.0  Comilla        Comilla  Chandpur     Hajigonj   
1             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
2             2001.0    3.0  Comilla        Comilla  Chandpur     Hajigonj   
3             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
4             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
..               ...    ...      ...            ...       ...          ...   
60            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
61            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
62            2002.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
63            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
64            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   

          lat        lon  EstimatedLoc  
0   23.254886  90.849722   interpolate  
1   23.257385  90.853146   interpolate  
2   23.261470  90.858600   interpolate  
3   23.262256  90.859989   interpolate  
4   23.263467  90.862129   interpolate  
..        ...        ...           ...  
60  24.590565  91.585901   interpolate  
61  24.590976  91.592102   interpolate  
62  24.590992  91.590435   interpolate  
63  24.591276  91.599333   interpolate  
64  24.591425  91.601015   interpolate  

[2385 rows x 22 columns]

In [18]:
df_cleaned_coord.isna().sum()

level_0              0
index                0
road                 0
km                   0
type                 0
LRPName              0
name                 0
length               0
condition            0
structureNr          0
roadName             0
chainage             0
width               18
constructionYear    18
spans               18
zone                 0
circle               0
division             0
sub-division         0
lat                  0
lon                  0
EstimatedLoc         0
dtype: int64

In this last and final iteration of cleaning, 11 bridges got removed. With this, the missing values got reduced to 17 entries

## Find out which bridge width corresponds to which amount of lanes
In order to make the probability of the bridges braking down in the model as accurate as possible, the amount of lanes each bridge has will be added based on the width of the bridge 

In [19]:
# Import the RMMS width files from the N1 and N2
N1_width = pd.read_table('../data/raw/N1.widths.processed.txt')
N2_width = pd.read_table('../data/raw/N2.widths.processed.txt')

In [20]:
N1_width

roadNo  roadId  startChainage  endChainage  width  nrLanes
0      N1    1585          0.000        0.130   29.7        8
1      N1    1585          0.130        0.400   14.0        4
2      N1    1585          0.400        1.050   28.3        8
3      N1    1585          1.050        5.100   16.5        5
4      N1    1585          5.100        6.200   16.2        5
..    ...     ...            ...          ...    ...      ...
59     N1    1585        429.682      437.968    5.8        2
60     N1    1585        437.968      444.511    5.7        2
61     N1    1585        444.511      451.519    5.6        2
62     N1    1585        451.519      459.326    5.7        2
63     N1    1585        459.326      462.254    6.3        2

[64 rows x 6 columns]

In [21]:
def find_min_max(dataframe, lanes):
    min, max = 100, 0 # pick numbers that will be overruled 
    df_lanes = dataframe[dataframe['nrLanes']==lanes]
    df_lanes.reset_index(inplace=True)
    for row in range(len(df_lanes)):
        if df_lanes.loc[row, 'width'] > max:
            max = df_lanes.at[row, 'width']
        if df_lanes.loc[row, 'width'] < min:
            min = df_lanes.at[row, 'width']
    return min, max

In [22]:
print('Information from N1 width files')
for lanes in range(9):
    min_lane, max_lane = find_min_max(N1_width, lanes)
    print(f'Lanes: {lanes} - min width: {min_lane} meters - max width: {max_lane} meters')

Information from N1 width files
Lanes: 0 - min width: 100 meters - max width: 0 meters
Lanes: 1 - min width: 100 meters - max width: 0 meters
Lanes: 2 - min width: 5.6 meters - max width: 8.2 meters
Lanes: 3 - min width: 10.3 meters - max width: 10.3 meters
Lanes: 4 - min width: 14.0 meters - max width: 15.0 meters
Lanes: 5 - min width: 15.3 meters - max width: 16.5 meters
Lanes: 6 - min width: 18.6 meters - max width: 46.9 meters
Lanes: 7 - min width: 100 meters - max width: 0 meters
Lanes: 8 - min width: 25.7 meters - max width: 29.7 meters


In [23]:
# There is an outlier in the 6 lanes maximum
N1_width[N1_width['nrLanes'] == 6] # so it has to be 18.9 meters

roadNo  roadId  startChainage  endChainage  width  nrLanes
18     N1    1585         22.330       22.800   46.9        6
26     N1    1585         41.761       45.819   18.9        6
27     N1    1585         45.819       54.029   18.9        6
28     N1    1585         54.029      229.046   18.8        6
30     N1    1585        230.606      232.754   18.6        6

In [24]:
print('Information from N2 width files')
for lanes in range(9):
    min_lane, max_lane = find_min_max(N2_width, lanes)
    print(f'Lanes: {lanes} - min width: {min_lane} meters - max width: {max_lane} meters')

Information from N2 width files
Lanes: 0 - min width: 100 meters - max width: 0 meters
Lanes: 1 - min width: 100 meters - max width: 0 meters
Lanes: 2 - min width: 7.0 meters - max width: 8.0 meters
Lanes: 3 - min width: 8.0 meters - max width: 12.0 meters
Lanes: 4 - min width: 13.0 meters - max width: 140.0 meters
Lanes: 5 - min width: 17.2 meters - max width: 17.2 meters
Lanes: 6 - min width: 18.0 meters - max width: 21.0 meters
Lanes: 7 - min width: 100 meters - max width: 0 meters
Lanes: 8 - min width: 100 meters - max width: 0 meters


In [25]:
# There is an outlier in the 4 lanes maximum
N2_width[N2_width['nrLanes'] == 4] # so it has to be 17.0 meters

roadNo  roadId  startChainage  endChainage  width  nrLanes
0       N2    1479          0.000        0.130   15.2        4
1       N2    1479          0.130        0.260   14.0        4
10      N2    1479         10.996       11.276   14.0        4
16      N2    1479         25.416       26.036   13.4        4
19      N2    1479         27.376       27.626   13.4        4
20      N2    1479         27.626       27.876   13.4        4
21      N2    1479         27.876       28.426   14.0        4
23      N2    1479         29.826       30.076   13.4        4
24      N2    1479         30.076       30.426  140.0        4
34      N2    1479         42.526       42.616   17.0        4
35      N2    1479         42.616       42.766   17.0        4
36      N2    1479         42.766       42.916   14.0        4
38      N2    1479         50.171       50.391   17.0        4
40      N2    1479         51.671       51.871   16.0        4
42      N2    1479         52.671       52.821   16.0        4
44      N2    1479         54.436       54.726   16.4        4
46      N2    1479         58.876       59.126   16.0        4
48      N2    1479         63.848       64.048   16.0        4
52      N2    1479         70.098       70.848   16.0        4
54      N2    1479         71.108       73.598   17.0        4
55      N2    1479         73.598       73.848   13.5        4
58      N2    1479         85.713       85.963   14.0        4
62      N2    1479         92.246       92.796   16.0        4
69      N2    1479        116.126      116.516   13.0        4
77      N2    1479        141.181      141.431   16.0        4
80      N2    1479        145.841      146.081   16.0        4
100     N2    1479        181.580      181.730   16.0        4
102     N2    1479        190.820      190.970   16.0        4
120     N2    1479        225.323      225.528   16.0        4

Based on the width values from the RMMS files, and the assumption that roads are always symmetrical, and the values from this official Bangladeshi source (https://www.rhd.gov.bd/Documents/RoadDesignAndSafety/Road%20Geometric%20Design%20Manual/Cross-Sections.pdf), which states that a roads needs to be 3.7m wide, we have landed on the following amount of lanes for width ranges <br>
<12m = 2 lanes <br>
12-18m = 4 lanes <br>
18-22m = 6 lanes <br>
22-30m = 8 lanes <br>

## Generate the amount of lanes per road

In [26]:
# For the bridges with missing values in the width column, check where on the road they are placed
df_cleaned_coord[df_cleaned_coord['width'].isna()]

level_0  index   road       km                   type  LRPName  \
9         14   2607   N204   16.284      RCC Girder Bridge  LRP019b   
18        18  16292   R203   15.057  Truss with Steel Deck  LRP015a   
21        18  11292  Z1031   18.998      RCC Girder Bridge  LRP019a   
16        24   2853   R241   39.086             RCC Bridge  LRP039a   
36        47   5619  Z1042   31.775      RCC Girder Bridge  LRP034a   
15        52   2281   R151   31.573  Truss with Steel Deck  LRP033d   
21        44  16188   R151   26.086  Truss with Steel Deck  LRP026a   
14       118  15498   R360  102.911       PC Girder Bridge  LRP103b   
1         96  15266   R170   55.306      RCC Girder Bridge  LRP054c   
75       150  15762     N2   97.263      RCC Girder Bridge  LRP098a   
89        72  15743     N2   58.035      RCC Girder Bridge  LRP058a   
335      390    772     N2  183.481       PC Girder Bridge  LRP184a   
387      454    823     N2  224.112       PC Girder Bridge  LRP225a   
30        21  17358  Z1048   10.195  Baily with Steel Deck  LRP011b   
38        20   2873   R250   32.593      RCC Girder Bridge  LRP033a   
507      174     89     N1  148.174       PC Girder Bridge  LRP149b   
624       21     14     N1   17.222       PC Girder Bridge  LRP018a   
629       14     11     N1   12.688       PC Girder Bridge  LRP013b   

                     name  length condition  structureNr  ... width  \
9       Ram Bhanga Bridge   50.00         A       120979  ...   NaN   
18    Ramchandradi Bridge  148.11         C       120134  ...   NaN   
21     Mandair pul Bridge    9.46         B       121846  ...   NaN   
16          Chayra Bridge   19.20         A       121842  ...   NaN   
36          Shalpa Bridge   36.60         A       120374  ...   NaN   
15         Gozaria Bridge   42.67         A       121390  ...   NaN   
21         Datmara Bridge   18.38         C       121817  ...   NaN   
14       Ganokkhal Bridge   43.45         B       120041  ...   NaN   
1    Hashiyar Para Bridge   15.80         B       121173  ...   NaN   
75        Khilsima Bridge   27.03         C       121822  ...   NaN   
89      Bridge Over Arial  241.61         C       121824  ...   NaN   
335       Minazpur Bridge   73.00         A       120959  ...   NaN   
387     Teli Bazar Bridge   40.80         A       120623  ...   NaN   
30           Gogor Bridge   15.25         C       120804  ...   NaN   
38      Kamar Gram Bridge   14.50         A       121823  ...   NaN   
507      Rampur Bridge(R)   22.15         A       121134  ...   NaN   
624  Darikandi Bridge (L)   20.50         A       119921  ...   NaN   
629    Madanpur Bridge(R)   27.50         A       119900  ...   NaN   

     constructionYear  spans        zone         circle      division  \
9                 NaN    NaN      Sylhet  Moulavi Bazar      Habiganj   
18                NaN    NaN       Dhaka          Dhaka   Narayanganj   
21                NaN    NaN     Comilla       Noakhali          Feni   
16                NaN    NaN      Sylhet         Sylhet     Sunamganj   
36                NaN    NaN     Comilla        Comilla  Brahmanbaria   
15                NaN    NaN  Chittagong     Chittagong    Chittagong   
21                NaN    NaN  Chittagong     Chittagong    Chittagong   
14                NaN    NaN  Mymensingh     Mymensingh   Kishoreganj   
1                 NaN    NaN  Chittagong     Chittagong      Dohazari   
75                NaN    NaN     Comilla        Comilla  Brahmanbaria   
89                NaN    NaN       Dhaka          Dhaka     Narsingdi   
335               NaN    NaN      Sylhet  Moulavi Bazar      Habiganj   
387               NaN    NaN      Sylhet         Sylhet        Sylhet   
30                NaN    NaN     Comilla        Comilla       Comilla   
38                NaN    NaN      Sylhet         Sylhet        Sylhet   
507               NaN    NaN     Comilla       Noakhali          Feni   
624               NaN    NaN       Dhaka          Dhaka   Naraya

In [27]:
# Based on the LRPName and the chainge, none of the brides with NaN values are placed at the beginning of the end of their respective roads. This makes it safe to fill the missing values using forward fill, as there is no risk of filling the bridge with values from another road
df_cleaned_coord.ffill()

level_0  index   road      km                   type  LRPName  \
0        70   5675  Z1044  39.882            Box Culvert  LRP040b   
1        69   5674  Z1044  39.444            Box Culvert  LRP040a   
2        68   5673  Z1044  38.698       PC Girder Bridge  LRP039c   
3        67   5672  Z1044  38.530            Box Culvert  LRP039b   
4        66   5671  Z1044  38.271            Box Culvert  LRP039a   
..      ...    ...    ...     ...                    ...      ...   
60       68  15365   R240  47.812            Box Culvert  LRP047e   
61       71   2844   R240  48.610            Box Culvert  LRP048c   
62       69  16421   R240  48.270  Baily with Steel Deck  LRP048a   
63       72  15366   R240  49.160            Box Culvert  LRP049a   
64       73   2845   R240  49.333            Box Culvert  LRP049c   

                             name  length condition  structureNr  ... width  \
0            HOZIGANG BOX CULVERT   12.30         A       102609  ...  7.20   
1               BODORPUR SUBMARGE    1.50         A       102607  ...  5.66   
2                 BODORPUR BRIDGE   64.60         A       102608  ...  9.40   
3        kathalia sub box culvert    1.50         A       102604  ...  5.66   
4                KATHALIA CULVERT    3.00         A       102602  ...  7.30   
..                            ...     ...       ...          ...  ...   ...   
60             KHURSI BOX CULVERT   12.23         B       102244  ...  9.75   
61             RAIPUR BOX CULVERT    1.50         A       102249  ...  8.50   
62  PAIPUR BAILEY WITH STEEL DECK   24.30         C       102247  ...  3.80   
63           AUSKANDI BOX CULVERT   12.30         B       102250  ...  9.75   
64           AUSKAMDI BOX CULVERT    1.50         A       102253  ...  8.50   

    constructionYear  spans     zone         circle  division sub-division  \
0             2001.0    2.0  Comilla        Comilla  Chandpur     Hajigonj   
1             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
2             2001.0    3.0  Comilla        Comilla  Chandpur     Hajigonj   
3             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
4             2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
..               ...    ...      ...            ...       ...          ...   
60            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
61            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
62            2002.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
63            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
64            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   

          lat        lon  EstimatedLoc  
0   23.254886  90.849722   interpolate  
1   23.257385  90.853146   interpolate  
2   23.261470  90.858600   interpolate  
3   23.262256  90.859989   interpolate  
4   23.263467  90.862129   interpolate  
..        ...        ...           ...  
60  24.590565  91.585901   interpolate  
61  24.590976  91.592102   interpolate  
62  24.590992  91.590435   interpolate  
63  24.591276  91.599333   interpolate  
64  24.591425  91.601015   interpolate  

[2385 rows x 22 columns]

In [28]:
def add_lanes(df):
    dataframe = df.copy()
    dataframe.drop(columns=['level_0', 'index'], inplace=True)
    dataframe.reset_index(inplace=True)
    dataframe.drop(columns=['index'], inplace=True)
    for row in range(len(dataframe)):
         if dataframe.loc[row, 'width'] < 12:
            dataframe.at[row, 'lanes'] = 2
         elif (dataframe.loc[row, 'width'] >= 12) & (dataframe.loc[row, 'width'] < 18):
            dataframe.at[row, 'lanes'] = 4
         elif (dataframe.loc[row, 'width'] >= 18) & (dataframe.loc[row, 'width'] < 22):
            dataframe.at[row, 'lanes'] = 6
         else:
            dataframe.at[row, 'lanes'] = 8
    return dataframe

In [29]:
cleaned_bridges_final = add_lanes(df_cleaned_coord)
cleaned_bridges_final

road      km                   type  LRPName  \
0     Z1044  39.882            Box Culvert  LRP040b   
1     Z1044  39.444            Box Culvert  LRP040a   
2     Z1044  38.698       PC Girder Bridge  LRP039c   
3     Z1044  38.530            Box Culvert  LRP039b   
4     Z1044  38.271            Box Culvert  LRP039a   
...     ...     ...                    ...      ...   
2380   R240  47.812            Box Culvert  LRP047e   
2381   R240  48.610            Box Culvert  LRP048c   
2382   R240  48.270  Baily with Steel Deck  LRP048a   
2383   R240  49.160            Box Culvert  LRP049a   
2384   R240  49.333            Box Culvert  LRP049c   

                               name  length condition  structureNr  \
0              HOZIGANG BOX CULVERT   12.30         A       102609   
1                 BODORPUR SUBMARGE    1.50         A       102607   
2                   BODORPUR BRIDGE   64.60         A       102608   
3          kathalia sub box culvert    1.50         A       102604   
4                  KATHALIA CULVERT    3.00         A       102602   
...                             ...     ...       ...          ...   
2380             KHURSI BOX CULVERT   12.23         B       102244   
2381             RAIPUR BOX CULVERT    1.50         A       102249   
2382  PAIPUR BAILEY WITH STEEL DECK   24.30         C       102247   
2383           AUSKANDI BOX CULVERT   12.30         B       102250   
2384           AUSKAMDI BOX CULVERT    1.50         A       102253   

                                               roadName  chainage  ...  \
0                         Gouripur-Kachua-Hajiganj Road    39.882  ...   
1                         Gouripur-Kachua-Hajiganj Road    39.444  ...   
2                         Gouripur-Kachua-Hajiganj Road    38.698  ...   
3                         Gouripur-Kachua-Hajiganj Road    38.530  ...   
4                         Gouripur-Kachua-Hajiganj Road    38.271  ...   
...                                                 ...       ...  ...   
2380  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    47.812  ...   
2381  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    48.610  ...   
2382  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    48.270  ...   
2383  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    49.160  ...   
2384  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...    49.333  ...   

      constructionYear  spans     zone         circle  division sub-division  \
0               2001.0    2.0  Comilla        Comilla  Chandpur     Hajigonj   
1               2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
2               2001.0    3.0  Comilla        Comilla  Chandpur     Hajigonj   
3               2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
4               2001.0    1.0  Comilla        Comilla  Chandpur     Hajigonj   
...                ...    ...      ...            ...       ...          ...   
2380            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
2381            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
2382            2002.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
2383            1998.0    2.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   
2384            1978.0    1.0   Sylhet  Moulavi Bazar  Habiganj     Habigonj   

            lat        lon  EstimatedLoc lanes  
0     23.254886  90.849722   interpolate   2.0  
1     23.257385  90.853146   interpolate   2.0  
2     23.261470  90.858600   interpolate   2.0  
3     23.262256  90.859989   interpolate   2.0  
4     23.263467  90.862129   interpolate   2.0  
...         ...        ...           ...   ...  
2380  24.590565  91.585901   interpolate   2.0  
2381  24.590976  91.592102   interpolate   2.0  
2382  24.590992  91.590435   interpolate   2.0  
2383  24.591276  91.599333   interpolate   2.0  
2384  24.591425  91.601015   interpolate   2.0  

[2385 rows x 21 columns]

In [30]:
cleaned_bridges_final.to_csv('../data/semi-processed/cleaned_bridges_final.csv')